
![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare_jsl/NER_JSL.ipynb)

# **Detect Clinical Entities**

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload license_keys.json to the folder that opens. Otherwise, you can look at the example outputs at the bottom of the notebook.

# **Colab Setup**

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install johnsnowlabs

In [ ]:
from google.colab import files
print("Please Upload your John Snow Labs License using the button below")
license_keys = files.upload()

In [ ]:
from johnsnowlabs import *

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect

jsl.install()

## Start Session

In [ ]:
from johnsnowlabs import *
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

In [ ]:
spark

# **Models:**


```
ner_jsl
ner_jsl_slim
ner_jsl_enriched
ner_jsl_greedy


bert_token_classifier_ner_jsl
bert_token_classifier_ner_jsl_slim

```



# **🔎Sample Text**

In [ ]:
sample_text = """The patient is a 40-year-old white male who presents with a chief complaint of "chest pain". The patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. He has been advised Aspirin 81 milligrams QDay, insulin 50 units in a.m. HCTZ 50 mg QDay. Nitroglycerin 1/150 sublingually PRN chest pain."""




# ***🔎For models:***

- ***ner_jsl***
- ***ner_jsl_slim***
- ***ner_jsl_enriched***
- ***ner_jsl_greedy***


### **Define Spark NLP pipeline**

In [ ]:
jsl_model_list = ["ner_jsl", 
                  "ner_jsl_slim", 
                  "ner_jsl_enriched", 
                  "ner_jsl_greedy"]

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
                .setInputCol("text")\
                .setOutputCol("document")

sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
                .setInputCols(["document"]) \
                .setOutputCol("sentence") 

tokenizer = nlp.Tokenizer()\
                .setInputCols(["sentence"])\
                .setOutputCol("token")

jsl_ner_converter = nlp.NerConverter() \
                .setInputCols(["sentence", "token", "jsl_ner"]) \
                .setOutputCol("ner_chunk")

embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
                .setInputCols(["sentence", "token"])\
                .setOutputCol("embeddings")
  
for model_name in jsl_model_list:

  jsl_ner = medical.NerModel.pretrained(model_name, "en", "clinical/models") \
                          .setInputCols(["sentence", "token", "embeddings"]) \
                          .setOutputCol("jsl_ner")


  jsl_ner_pipeline = Pipeline(stages=[documentAssembler, 
                                      sentenceDetector,
                                      tokenizer,
                                      embeddings,
                                      jsl_ner,
                                      jsl_ner_converter])


  jsl_ner_model = jsl_ner_pipeline.fit(spark.createDataFrame([['']]).toDF("text"))
  light_model = LightPipeline(jsl_ner_model)
  light_result = light_model.fullAnnotate(sample_text)

  print("\n\n\n")
  print(f"***************  The visualization results for {model_name} ***************")
  print("\n\n\n")

  from sparknlp_display import NerVisualizer
  visualiser = NerVisualizer()
  visualiser.display(light_result[0], label_col='ner_chunk', document_col='document')
  print("\n\n\n")

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_jsl download started this may take some time.
[OK!]




***************  The visualization results for ner_jsl ***************










ner_jsl_slim download started this may take some time.
[OK!]




***************  The visualization results for ner_jsl_slim ***************










ner_jsl_enriched download started this may take some time.
[OK!]




***************  The visualization results for ner_jsl_enriched ***************










ner_jsl_greedy download started this may take some time.
[OK!]




***************  The visualization results for ner_jsl_greedy ***************








# ***🔎For models:***

- ***bert_token_classifier_ner_jsl***
- ***bert_token_classifier_ner_jsl_slim***



### **Define Spark NLP pipeline**

In [ ]:
bert_jsl_ner_model_list = ["bert_token_classifier_ner_jsl", 
                           "bert_token_classifier_ner_jsl_slim"]

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

sentenceDetector = nlp.SentenceDetectorDLModel.pretrained() \
        .setInputCols(["document"]) \
        .setOutputCol("sentence") 

tokenizer = nlp.Tokenizer()\
        .setInputCols("sentence")\
        .setOutputCol("token")

ner_converter = nlp.NerConverter()\
        .setInputCols(["sentence","token","ner"])\
        .setOutputCol("ner_chunk")

for model_name in bert_jsl_ner_model_list:
  tokenClassifier = medical.BertForTokenClassifier.pretrained(model_name, "en", "clinical/models")\
        .setInputCols(["token", "sentence"])\
        .setOutputCol("ner")\
        .setCaseSensitive(True)

  bert_jsl_ner_pipeline =  Pipeline(stages=[documentAssembler, 
                                            sentenceDetector, 
                                            tokenizer, 
                                            tokenClassifier, 
                                            ner_converter])


  bert_jsl_ner_model = bert_jsl_ner_pipeline.fit(spark.createDataFrame([['']]).toDF("text"))
  light_model = LightPipeline(bert_jsl_ner_model)
  light_result = light_model.fullAnnotate(sample_text)

  print("\n\n\n")
  print(f"***************  The visualization results for {model_name} ***************")
  print("\n\n\n")

  from sparknlp_display import NerVisualizer
  visualiser = NerVisualizer()
  visualiser.display(light_result[0], label_col='ner_chunk', document_col='document')
  print("\n\n\n")

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
bert_token_classifier_ner_jsl download started this may take some time.
[OK!]




***************  The visualization results for bert_token_classifier_ner_jsl ***************










bert_token_classifier_ner_jsl_slim download started this may take some time.
[OK!]




***************  The visualization results for bert_token_classifier_ner_jsl_slim ***************




